In [29]:
from model import Resnet
level_id = 4

In [30]:
from utils import split_data
import torch
test_cifar_data = torch.ones(2, 3, 32, 32)
x_a, x_b = split_data(test_cifar_data, 'cifar10')
print(x_a.shape)

torch.Size([2, 3, 32, 16])


In [31]:
test_data = torch.ones(2, 3, 64, 64)
client, server = Resnet(level=level_id)
output = client(test_data)
print(output.shape)


torch.Size([2, 256, 4, 4])


In [32]:
decoder_input = output
print(decoder_input.shape)

torch.Size([2, 256, 4, 4])


In [33]:
from model import cifar_discriminator_model, resnet_discriminator
d = resnet_discriminator(decoder_input.shape[1], level=level_id)

In [34]:
d_output = d(output)
print(d_output.shape)

torch.Size([2, 1])


In [35]:
from model import resent_decoder
d_input_shape = output.shape[1:]
decoder = resent_decoder(d_input_shape, level=level_id)
decoder_output = decoder(decoder_input)
print(decoder_output.shape)

[DECODER] activation:  None
torch.Size([2, 3, 64, 64])
